In [1]:
#!/usr/bin python2.7

from astropy.io       import fits
from astropy.table    import Table
from astropy.time     import Time
from datetime         import date
from datetime         import datetime
import ephem
import fileinput
import getpass
import math
import matplotlib.pyplot                   as plt
import matplotlib                          as mpl
import numpy                               as np
from numpy            import linalg        as LA
import os
import pyfits
import scipy.optimize                      as optimization
from scipy            import integrate
from scipy            import interpolate
from scipy.optimize   import curve_fit
import subprocess
from subprocess       import Popen, PIPE
import urllib2
from work_module      import calculate
from work_module      import detector
from work_module      import readfile
from work_module      import writefile
calc = calculate()
det = detector()
rf = readfile()
wf = writefile()

In [7]:
day = 150926
#day = np.array([150927, 150810, 151120, 151126])
detectors = np.array(['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'na'])
#detector = 'n5'
data_type = 'ctime'

In [8]:
def get_coeffs(day, detector_name, data_type = 'ctime'):
    """This function reads the fitting coefficients for the plot-fits-files and saves them in arrays of the form: cgb, magnetic, earth_ang, sun_ang, pointsources, echans\n
        Input:\n
        calculate.get_coeffs( day = YYMMDD, detector (f. e. 'n5'), data_type = 'ctime' (or 'cspec'))\n
        Output:\n
        0 = cgb\n
        1 = magnetic\n
        2 = earth_ang\n
        3 = sun_ang\n
        4 = pointsources\n
        5 = echans"""
    
    #sources_data = readfile.point_sources(readfile())
    sources_data = rf.point_sources()
    sources_names = sources_data[0]
    sources_coordinates = sources_data[1]
    sources_number = len(sources_names)
    
    user = getpass.getuser()
    
    if data_type == 'ctime':
        echans = np.arange(8)
    elif data_type == 'cspec':
        echans = np.arange(128)
    else:
        print 'Invalid data type: ' + str(data_type) + '\n Please insert an appropriate data type (ctime or cspec).'
        return
    
    cgb = []
    magnetic = []
    earth_ang = []
    sun_ang = []
    sources_coeff = []
    
    for echan in echans:
        print echan
    #See if Fit has already been calculated before
        if data_type == 'ctime':
            fitsname_data = 'ctime_' + detector_name + '_e' + str(echan) + '_kt.fits'
        else:
            fitsname_data = 'cspec_' + detector_name + '_e' + str(echan) + '__kt.fits'
    
        fits_path_data = '/home/' + user + '/Work/Fits_data/' + str(day) + '/'
        if not os.access(fits_path_data, os.F_OK):
            os.mkdir(fits_path_data)
        fitsfilepath_data = os.path.join(fits_path_data, fitsname_data)
        
        if os.path.isfile(fitsfilepath_data) == True:
            #data = readfile.fits_data (readfile(), day, detector_name, echan, data_type)
            data = rf.fits_data (day, detector_name, echan, data_type)
            fit_coeffs = data[10].astype(np.float)
            cgb1 = fit_coeffs[0]
            magnetic1 = fit_coeffs[1]
            earth_ang1 = fit_coeffs[2]
            sun_ang1 = fit_coeffs[3]
            sources_names = data[12]
            sources_coeff1 = data[13].astype(np.float)
            sources_coeff1 = sources_coeff1[sources_coeff1 != 0]
        else:
            print 'The file does not exist for the energy-channel ' + str(echan) + '. Fit the data first or review your input.'
            return
        
        cgb.append(cgb1)
        magnetic.append(magnetic1)
        earth_ang.append(earth_ang1)
        sun_ang.append(sun_ang1)
        sources_coeff.append(sources_coeff1)
        
    cgb = np.array(cgb)
    magnetic = np.array(magnetic)
    earth_ang = np.array(earth_ang)
    sun_ang = np.array(sun_ang)
    pointsources = [sources_names, map(list, zip(*sources_coeff))]
    
    return cgb, magnetic, earth_ang, sun_ang, pointsources, echans

In [9]:
cgb_all = []
magnetic_all = []
earth_all = []
sun_all = []
for detector in detectors:
    print detector
    
    data = get_coeffs(day, detector)
    #print data
    cgb = data[0]
    magnetic = data[1]
    earth_ang = data[2]
    sun_ang = data[3]
    pointsources = data[4]
    echans = data[5]
    
    cgb_norm = cgb/cgb[1]
    magnetic_norm = magnetic/magnetic[3]
    earth_ang_norm = earth_ang/earth_ang[1]
    sun_ang_norm = sun_ang/sun_ang[1]
    
    cgb_all.append(cgb_norm)
    magnetic_all.append(magnetic_norm)
    earth_all.append(earth_ang_norm)
    sun_all.append(sun_ang_norm)

cgb_all = np.array(cgb_all)
magnetic_all = np.array(magnetic_all)
earth_all = np.array(earth_all)
sun_all = np.array(sun_all)

#print cgb_all
#print magnetic_all
#print earth_all
#print sun_all

n0
0
1
2
3
4
5
6
7
n1
0
1
2
3
4
5
6
7
n2
0
1
2
3
4
5
6
7
n3
0
1
2
3
4
5
6
7
n4
0
1
2
3
4
5
6
7
n5
0
1
2
3
4
5
6
7
n6
0
1
2
3
4
5
6
7
n7
0
1
2
3
4
5
6
7
n8
0
1
2
3
4
5
6
7
n9
0
1
2
3
4
5
6
7
na
0
1
2
3
4
5
6
7
[[ 0.22014369  1.          0.64347845  0.49132297  0.39810337  0.04681629
   0.00597156 -0.11072984]
 [ 0.15299201  1.          0.64617887  0.43731774  0.3631998   0.06471921
   0.01594566  0.09483812]
 [ 0.15217521  1.          0.60362085  0.41208668  0.34653497  0.06986325
   0.06848268  0.04965121]
 [ 0.16560251  1.          0.59723311  0.40851522  0.31918743  0.04697007
   0.02514208  0.07079961]
 [ 0.12035019  1.          0.61342582  0.40981435  0.33013395  0.06837575
   0.10977371  0.10274705]
 [ 0.18324359  1.          0.60019625  0.39547125  0.35317369  0.07351294
   0.15553203  0.0704868 ]
 [ 0.19804032  1.          0.60766183  0.47360994  0.33936657  0.03001607
   0.06645911 -0.01274227]
 [ 0.18345307  1.          0.59431765  0.40400798  0.30556446  0.0444661
   0.036665

In [28]:
echans_energy = np.array([10.76, 20.415, 38.803, 76.37, 190.19, 410.91, 751.94, 1466.43])
echans = np.array(['e0', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7'])

In [29]:
x = np.array([8, 19, 38, 75, 198, 417, 762, 1492])
cgb = np.array([5.49, 2.69, 1., 0.443, 0.164, 0.0185, 0.00600, 0.00212])
earth = np.array([0.0007377, 0.1017, 1., 1.205, 0.988, 0.301, 0.1943, 0.1436])
earth_occ = -np.array([7.00629318, 3.4035806, 1., 0.23188162, -0.05829884, -0.05086629, -0.03695036, -0.02970391])

energy_eff_width = np.array([0.138, 0.953, 1., 1.010, 0.859, 0.266, 0.109, 0.050])
energy_eff_area = np.array([0.042, 0.602, 1., 2.202, 7.047, 2.0758, 2.049, 2.139])
energy_eff_total_area = np.array([0.0377, 0.542, 1., 2.17, 7.77, 6.38, 8.48, 13.21])

cgb_eff_width = cgb*energy_eff_width
earth_occ_eff_width = earth_occ*energy_eff_width
cgb_eff_area = cgb*energy_eff_area
earth_occ_eff_area = earth_occ*energy_eff_area
cgb_eff_total_area = cgb*energy_eff_total_area
earth_occ_eff_total_area = earth_occ*energy_eff_total_area




In [ ]:
earth_ang = 0.
eff_area_frac = []
for i in range(0, 8):
    data = calc.earth_occ_eff(earth_ang, i)
    eff_area_frac.append(data)
    
eff_area_frac = np.array(eff_area_frac)
area_norm = eff_area_frac/eff_area_frac[2]

In [ ]:
print area_norm

In [27]:
### print cgb ###
#print 'n0: ', cgb_all[0]
#print 'n1: ', cgb_all[1]
#print 'n2: ', cgb_all[2]
#print 'n3: ', cgb_all[3]
#print 'n4: ', cgb_all[4]
#print 'n5: ', cgb_all[5]
#print 'n6: ', cgb_all[6]
#print 'n7: ', cgb_all[7]
#print 'n8: ', cgb_all[8]
#print 'n9: ', cgb_all[9]
#print 'na: ', cgb_all[10]
#print 'nb: ', cgb_all[11]

### plot cgb ###
mpl.rcParams['axes.color_cycle'] = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

for i in range(0, len(cgb_all)):
    plt.plot(echans_energy, cgb_all[i]/cgb_all[i,2], label = detectors[i])
    
plt.plot(x, cgb_eff_total_area, 'b--', label = 'total')
plt.plot(x, cgb_eff_area, 'r--', label = 'peak')
plt.plot(x, cgb_eff_width, 'm--', label = 'width')

plt.legend()
plt.show()

In [26]:
### print magnetic ###
#print 'n0: ', magnetic_all[0]
#print 'n1: ', magnetic_all[1]
#print 'n2: ', magnetic_all[2]
#print 'n3: ', magnetic_all[3]
#print 'n4: ', magnetic_all[4]
#print 'n5: ', magnetic_all[5]
#print 'n6: ', magnetic_all[6]
#print 'n7: ', magnetic_all[7]
#print 'n8: ', magnetic_all[8]
#print 'n9: ', magnetic_all[9]
#print 'na: ', magnetic_all[10]
#print 'nb: ', magnetic_all[11]

### plot magnetic ###
mpl.rcParams['axes.color_cycle'] = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

for i in range(0, len(magnetic_all)):
    plt.plot(echans_energy, magnetic_all[i]/magnetic_all[i,4], label = detectors[i])
    
plt.legend()
plt.show()

In [25]:
### print earth ###
#print 'n0: ', earth_all[0]
#print 'n1: ', earth_all[1]
#print 'n2: ', earth_all[2]
#print 'n3: ', earth_all[3]
#print 'n4: ', earth_all[4]
#print 'n5: ', earth_all[5]
#print 'n6: ', earth_all[6]
#print 'n7: ', earth_all[7]
#print 'n8: ', earth_all[8]
#print 'n9: ', earth_all[9]
#print 'na: ', earth_all[10]
#print 'nb: ', earth_all[11]

### plot earth ###
mpl.rcParams['axes.color_cycle'] = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

for i in range(0, len(earth_all)):
    plt.plot(echans_energy, earth_all[i]/earth_all[i,2], label = detectors[i])
    
plt.plot(x, earth_occ_eff_total_area, 'b--', label = 'Earth occultation')
plt.plot(x, earth_occ_eff_area, 'r--', label = 'Earth occultation')
plt.plot(x, earth_occ_eff_width, 'm--', label = 'Earth occultation')

plt.legend()
plt.show()

In [23]:
### print sun ###
#print 'n0: ', sun_all[0]
#print 'n1: ', sun_all[1]
#print 'n2: ', sun_all[2]
#print 'n3: ', sun_all[3]
#print 'n4: ', sun_all[4]
#print 'n5: ', sun_all[5]
#print 'n6: ', sun_all[6]
#print 'n7: ', sun_all[7]
#print 'n8: ', sun_all[8]
#print 'n9: ', sun_all[9]
#print 'na: ', sun_all[10]
#print 'nb: ', sun_all[11]

### plot sun ###
mpl.rcParams['axes.color_cycle'] = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

for i in range(0, len(sun_all)):
    plt.plot(echans_energy, sun_all[i]/sun_all[i,1], label = detectors[i])
    
plt.legend()
plt.show()

In [ ]:
###normalization###

#cgb_norm = cgb/cgb[2]
#magnetic_norm = magnetic/magnetic[2]
#earth_ang_norm = earth_ang/earth_ang[2]
#sun_ang_norm = sun_ang/sun_ang[2]
#echans_energy = np.array([8, 19, 38, 75, 198, 417, 762, 1492])
echans_energy = np.array([19, 38, 75, 198, 417, 762, 1492])

#pointsources_coeffs_norm = []
#for i in range(0, len(pointsources[1])):
#    coeff_ar = np.array(pointsources[1][i])
#    coeff_norm = coeff_ar/coeff_ar[2]
#    pointsources_coeffs_norm.append(coeff_norm)

#pointsources_coeffs_norm = np.array(pointsources_coeffs_norm)

In [ ]:
plt.plot(echans, cgb, 'b-', label = 'cgb')
plt.plot(echans, magnetic, 'r-', label = 'magn')
plt.plot(echans, earth_ang, 'g-', label = 'earth')
plt.plot(echans, sun_ang, 'y-', label = 'sun')
index = 0
plt.plot(echans, pointsources[1][index], 'k-', label = pointsources[0][index])

plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.plot(echans, cgb_norm, 'b-', label = 'cgb')
plt.plot(echans, magnetic_norm, 'r-', label = 'magn')
plt.plot(echans, earth_ang_norm, 'g-', label = 'earth')
plt.plot(echans, sun_ang_norm, 'y-', label = 'sun')
index = 0
plt.plot(echans, pointsources_coeffs_norm[index], 'k-', label = pointsources[0][index])

plt.grid()
plt.legend()
plt.show()